#Primeiro modelo Ebettha - Sistema de compatibilidade

No primeiro modelo aqui apresentado, a equipe optou por fazer um recorte da base de dados para observar como os resultados seriam mostrados e, após validação junto ao parceiro, buscar a generalização.

Dessa forma, o grupo utilizou somente os resultados de superfit dos candidatos e gestores e, para iniciar o código, optamos por utilizar somente dados de 6 usuários e 3 gestores, pois isso permitirá uma melhor visualização e interpretação dos resultados.

Abaixo, é possível verificar a construção desse modelo.

In [16]:
#Nessa célula, estamos somente importando as bibliotecas a serem utilizadas
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [13]:
#Nesta célula, fazemos o download das bases de dados para que seja possível replicar o código em qualquer máquina
!gdown 14DS1ZYhQ7JpLq-hYPQ3f95XTtB5_EkP6 --output candidatos.csv #Tabela de candidatos
!gdown 1xIzSoerJysUxJI1bAeWzQ2AzLx8PraLI --output gestores.csv #Tabela de gestores

Downloading...
From: https://drive.google.com/uc?id=14DS1ZYhQ7JpLq-hYPQ3f95XTtB5_EkP6
To: /content/candidatos.csv
100% 313/313 [00:00<00:00, 960kB/s]
Downloading...
From: https://drive.google.com/uc?id=1xIzSoerJysUxJI1bAeWzQ2AzLx8PraLI
To: /content/gestores.csv
100% 216/216 [00:00<00:00, 884kB/s]


In [14]:
#Nesta célula, criamos os dataframes apartir das tabelas que nós fizemos o download anteriormente
df_candidatos = pd.read_csv('candidatos.csv')
df_gestores = pd.read_csv('gestores.csv')

Como já mencionado, iremos utilizar somente as competências avaliadas no teste "genius" (superfit). Dessa forma, vamos analisar somente as competências descritas abaixo:

In [15]:
# Habilidades para considerar
habilidades = ['superfit_dis', 'superfit_sin', 'superfit_cur', 'superfit_int', 'superfit_eng', 'superfit_res']

In [17]:
#Nesta célula, iremos transformar os nossos dataframes em matrizes numpy, pois iremos fazer algumas operações que necessitam desse tipo
candidatos = df_candidatos[habilidades].to_numpy()
gestores = df_gestores[habilidades].to_numpy()

In [19]:
#Aqui, calculamos a matriz de distância euclidiana tendo como conjunto as matrizes de gestores e candidatos.
distancias = cdist(candidatos, gestores, 'euclidean')

A célula anterior é de suma importância para o nosso modelo, pois é com ela que teremos base para construir a nossa metrificação que servirá para definir o nível de compatibilidade entre candidato e gestores. Nas células a seguir será possível entender como esse cálculo é feito.

In [22]:
# Calcula a distância máxima possível baseado na variação dos dados
diferenca_max = np.ptp(np.vstack([candidatos, gestores]), axis=0)
distancia_max_possivel = np.sqrt(np.sum(diferenca_max ** 2))

# Calculando a compatibilidade
compatibilidade = (1 - distancias / distancia_max_possivel) * 100

Considerando que os dados já estão normalizados e vão de 0 até 1, podemos fazer algumas considerações sobre a fórmula apresentada acima:


*   O valor máximo para distancia_max_possivel é um, que é o caso de quanto o gestor e o candidato são considerados completamente opostos.
*   O valor de distancia varia a cada candidato sendo que, caso a distância seja 1 (o mais diferente possível), o resultado da fórmula de compatibilidade é 0.
*   Por outro lado, caso o gestor e o candidato sejam exatamente iguais, o valor de distancias será 0, e com isso, o resultado da compatibilidade será 100%.



Por fim, na célula abaixo, podemos enfim verificar o nível de compatibilidade entre gestores e candidatos.

In [21]:
# Classificar gestores para cada candidato
for i in range(len(candidatos)):
    ranking = np.argsort(compatibilidade[i])  # Classifica os índices com base na compatibilidade
    ranking_reverso = ranking[::-1]  # Inverte para obter do maior para o menor
    print(f"Ranking de gestores para o Candidato {i+1} (ID do Candidato: {df_candidatos.loc[i, 'id']})")

    for j, gestor_index in enumerate(ranking_reverso):
        print(f"  {j+1}. Gestor com ID {df_gestores.loc[gestor_index, 'id']} com {compatibilidade[i][gestor_index]:.2f}% de compatibilidade.")

Ranking de gestores para o Candidato 1 (ID do Candidato: 1)
  1. Gestor com ID 4 com 100.00% de compatibilidade.
  2. Gestor com ID 3 com 100.00% de compatibilidade.
  3. Gestor com ID 1 com 53.95% de compatibilidade.
  4. Gestor com ID 2 com 39.48% de compatibilidade.
Ranking de gestores para o Candidato 2 (ID do Candidato: 2)
  1. Gestor com ID 2 com 75.79% de compatibilidade.
  2. Gestor com ID 1 com 71.89% de compatibilidade.
  3. Gestor com ID 4 com 45.43% de compatibilidade.
  4. Gestor com ID 3 com 45.43% de compatibilidade.
Ranking de gestores para o Candidato 3 (ID do Candidato: 3)
  1. Gestor com ID 1 com 85.88% de compatibilidade.
  2. Gestor com ID 2 com 74.86% de compatibilidade.
  3. Gestor com ID 4 com 51.27% de compatibilidade.
  4. Gestor com ID 3 com 51.27% de compatibilidade.
Ranking de gestores para o Candidato 4 (ID do Candidato: 4)
  1. Gestor com ID 1 com 32.60% de compatibilidade.
  2. Gestor com ID 2 com 25.72% de compatibilidade.
  3. Gestor com ID 4 com 23.86

##Ponto de melhoria

Durante a construção desse modelo, foi verificado que, caso existe um candidato com habilidades muito boas, melhores que a dos gestores, o nosso código classifica ele com pouca compatibilidade, sendo que, na vida real, um candidato muito bom deveria ser compatível com a vaga. Este é um problema que ainda está em discussão sobre formas de contorná-lo.

Outro ponto a se observar é que este é só um protótipo que será utilizado para validação junto ao cliente. Na próxima sprint (sprint 4), tanto esse modelo como outros dois modelos candidatos estarão finalizados e testados, sendo possível escolher o melhor.